In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [2]:
inpatient_specialty = pd.read_csv('/home/bhatti/dataset/VCHAMPS/inpatient_specialty_train.csv').iloc[:,1:]

In [3]:
inpatient_specialty

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State
0,1,78.725684,2022-12-31 05:41:51.0,2023-01-01 23:06:29.0,DERMATOLOGY,Indiana
1,100001,86.568969,2012-07-21 23:00:00.0,2012-07-22 19:18:47.0,PM&R TRANSITIONAL REHAB,North Carolina
2,100015,56.646451,2002-07-16 06:40:13.0,2002-07-16 06:40:13.0,INTERMEDIATE MEDICINE,Massachusetts
3,100015,58.247495,2004-02-21 11:01:35.0,2004-02-22 14:56:40.0,DOMICILIARY PTSD,Rhode Island
4,100019,81.174959,1999-10-24 10:44:24.0,1999-10-31 10:20:31.0,PLASTIC SURGERY,Florida
...,...,...,...,...,...,...
1452905,99993,58.013747,2007-02-18 00:35:16.0,2007-02-18 04:55:16.0,METABOLIC,Kentucky
1452906,99994,83.090126,2016-10-18 10:35:16.0,2016-10-21 11:02:57.0,NH SHORT STAY DEMENTIA CARE,Texas
1452907,99994,85.951737,2019-08-30 08:38:03.0,2019-09-05 09:28:48.0,"PULMONARY, NON-TB",Texas
1452908,99996,55.942565,2011-08-17 00:10:52.0,2011-08-17 00:10:52.0,GENERAL(ACUTE MEDICINE),Georgia


In [4]:
pd.isnull(inpatient_specialty).sum() 

Internalpatientid         0
Age at specialty          0
Specialty start date      0
Specialty end date      437
Specialty                 0
State                     0
dtype: int64

In [5]:
inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])]

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State
699,101476,74.957130,2019-12-03 19:56:26.0,NaN,HOSPICE FOR ACUTE CARE,Pennsylvania
1044,102152,87.421539,2024-04-11 13:44:24.0,NaN,DOMICILIARY SUBSTANCE ABUSE,Missouri
9216,104121,79.251161,2023-09-14 07:10:49.0,NaN,METABOLIC,Texas
13064,100569,70.366120,2019-05-21 19:36:23.0,NaN,SPINAL CORD INJURY OBSERVATION,Nevada
14156,109098,53.250477,2021-04-22 01:11:08.0,NaN,NH SHORT STAY RESTORATIVE,California
...,...,...,...,...,...,...
1431229,87364,79.014826,2023-01-19 22:03:04.0,NaN,PTSD RESIDENTIAL REHAB PROG,California
1432593,93641,74.926870,2023-01-14 18:30:01.0,NaN,OB/GYN,Not specified (no value)
1438157,91467,61.037133,2021-03-23 01:43:35.0,NaN,CARDIAC-STEP DOWN UNIT,Missouri
1442374,98483,73.038032,2024-02-10 09:37:03.0,NaN,UROLOGY,Minnesota


# Preprocess Time
Calculate period mean and std values.

In [6]:
inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])][inpatient_specialty['Internalpatientid'] == 98424]

/tmp/ipykernel_243841/1186491515.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  inpatient_specialty[pd.isnull(inpatient_specialty['Specialty end date'])][inpatient_specialty['Internalpatientid'] == 98424]


,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State
1449822,98424,75.971028,2024-08-03 09:51:23.0,NaN,MEDICAL ICU,California


In [7]:
inpatient_specialty['Specialty start date'] = inpatient_specialty['Specialty start date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['Specialty end date'] = inpatient_specialty['Specialty end date'].apply(preprocess_data.preprocess_time_string)
inpatient_specialty['period'] = preprocess_data.get_period(inpatient_specialty['Specialty start date'],inpatient_specialty['Specialty end date'])

In [8]:
period_mean = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))

/tmp/ipykernel_243841/2218735275.py:2: RuntimeWarning: Mean of empty slice
  period_mean['period'] = period_mean['period'].apply(lambda x: np.nanmean(x.total_seconds()/86400))


In [9]:
period_std = inpatient_specialty.groupby('Internalpatientid')['period'].agg(lambda x: x).reset_index()
period_std['period'] = period_std['period'].apply(lambda x: np.nanstd(x.total_seconds()/86400,ddof=0))

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [10]:
period_mean = period_mean.rename(columns={'period': 'period mean'})
period_std = period_std.rename(columns={'period': 'period std'})

# Preprocess Specialty
Then consider the specialty, since it has a lot of different values, we want to simplify it in to several classes.
By studying the values, we can have the classify it into Medical Specialties (encoded as 0) and Rehabilitation and Support Programs (encoded as 1). And finally impute the no specific value (3.75%) with imputate_nan_binary.

In [11]:
specialty_class = preprocess_data.generate_dict_from_csv('/home/vivi/FDA_datasets/specialty_class.csv','Specialty','Class')

In [12]:
inpatient_specialty['Specialty encoded'] = preprocess_data.preprocess_substitute_categories(specialty_class, 
                                                                               inpatient_specialty['Specialty'])

In [13]:
specialty_count = inpatient_specialty.groupby('Internalpatientid')['Specialty'].count().reset_index()

In [14]:
specialty_count = specialty_count.rename(columns={'Specialty': 'specialty count'})

In [15]:
specialty_medical_count = inpatient_specialty.groupby('Internalpatientid')['Specialty encoded'].agg(lambda x:  x.eq(0).sum()).reset_index()

In [16]:
specialty_support_count = inpatient_specialty.groupby('Internalpatientid')['Specialty encoded'].agg(lambda x:  x.eq(1).sum()).reset_index()

In [17]:
specialty_medical_count = specialty_medical_count.rename(columns={'Specialty encoded': 'specialty medical count'})
specialty_support_count = specialty_support_count.rename(columns={'Specialty encoded': 'specialty support count'})

In [18]:
pd.isnull(period_std).sum() 

Internalpatientid    0
period std           2
dtype: int64

In [19]:
period_std[pd.isnull(period_std['period std'])]

,Internalpatientid,period std
17983,35865,NaN
81462,162785,NaN


In [20]:
inpatient_specialty[inpatient_specialty['Internalpatientid'] == 35865]

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded
804968,35865,104.054027,2022-12-01 17:55:27,NaT,EVAL/BRF TRMT PTSD UNIT(EBTPU),Missouri,NaT,1.0


In [21]:
inpatient_specialty[inpatient_specialty['Internalpatientid'] == 162785]

,Internalpatientid,Age at specialty,Specialty start date,Specialty end date,Specialty,State,period,Specialty encoded
535931,162785,77.224569,2020-02-27 13:55:25,NaT,SURGICAL ICU,Georgia,NaT,0.0


In [22]:
period_mean[pd.isnull(period_mean['period mean'])]

,Internalpatientid,period mean
17983,35865,NaN
81462,162785,NaN


# Preprocess Age

In [23]:
age_mean = inpatient_specialty.groupby('Internalpatientid')['Age at specialty'].mean().reset_index()
age_mean = age_mean.rename(columns={'Age at specialty': 'mean age at specailty'})

# Merge All

In [24]:
listTables = [period_mean, period_std, specialty_medical_count,specialty_support_count, specialty_count ]
inpatient_specialty_preprocessed = age_mean.copy()
for i in listTables:
    inpatient_specialty_preprocessed = inpatient_specialty_preprocessed.merge(i,how = 'left', on = 'Internalpatientid')


In [25]:
inpatient_specialty_preprocessed

,Internalpatientid,mean age at specailty,period mean,period std,specialty medical count,specialty support count,specialty count
0,1,71.760299,1.985007,2.286435,8,0,8
1,2,65.896816,1.695425,3.832509,51,3,55
2,3,79.941238,81.500222,162.085913,1,4,5
3,4,84.177706,2.383800,1.709373,3,0,3
4,5,76.469781,1.407809,1.351702,3,3,6
...,...,...,...,...,...,...,...
84538,169055,58.918427,1.617095,1.735104,1,3,4
84539,169057,82.335103,1.277754,2.100649,58,19,77
84540,169060,65.595637,1.681511,3.024752,20,7,27
84541,169062,72.780802,6.870090,11.462599,20,6,26


In [26]:
inpatient_specialty_preprocessed = inpatient_specialty_preprocessed.dropna(subset=['period mean'])

In [27]:
inpatient_specialty_preprocessed.to_csv('/home/vivi/FDA_datasets/inpatient_specialty_preprocessed.csv')

In [28]:
pd.isnull(inpatient_specialty_preprocessed).sum()

Internalpatientid          0
mean age at specailty      0
period mean                0
period std                 0
specialty medical count    0
specialty support count    0
specialty count            0
dtype: int64